Наши данные

In [15]:
import pandas as pd
import re

pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Continental_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Комментарии,Наименование,Модель,Ширина шины,Высота профиля,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,96794,NaN,8101,NaN,7317,NaN,[CONTISPORTCONTACT 5 SUV 23555R19 105W XL FR L...,SportContact 5 SUV,235,55,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/527704...,[8101],2023-07-29 18:50:34,NaN,1768586022
1,61852,price,8101,NaN,7317,NaN,125/80 R13 Continental ContiWinterContact TS80...,WinterContact TS800,125,80,...,355286,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/dd6/a72e54...,"[8101, 1403, 350201]",2023-08-09 14:10:08,NaN,1777012756
2,96794,NaN,8101,NaN,7317,NaN,14/155/65 Continental IceContact 3 TA 75T,IceContact 3,155,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/449302...,[8101],2023-07-29 18:50:34,NaN,1754590109
3,61852,NaN,8101,NaN,7317,NaN,14/185/60 Continental IceContact 3 TA 82T,IceContact 3,185,60,...,347594,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/72a/cf125c...,[8101],2023-07-07 14:50:12,NaN,1755712962
4,96794,NaN,8101,NaN,7317,NaN,145/65 R15 72T Continental VikingContact 7,VikingContact 7,145,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/136080...,"[8101, 829805]",2023-05-05 05:06:00,NaN,1694032869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20276,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/19 W 100 ContiWinterContac...,WinterContact TS830,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,"[8101, 2110, 7502, 5999]",2023-05-02 11:20:06,NaN,1694349152
20277,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/21 V 102 WinterContact TS ...,WinterContact TS860,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/765032...,"[8101, 7502]",2023-05-02 17:08:57,NaN,1695507907
20278,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/40/19 V 108 ContiWinterContac...,WinterContact TS830,295,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,[8101],2023-06-08 16:53:29,NaN,1694665532
20279,96794,NaN,8101,NaN,7317,NaN,Континенталь 305/40/20 Y 112 ContiSportContact...,SportContact 5,305,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/405568...,"[8101, 8398, 719825]",2023-05-02 17:08:57,NaN,1695320371


In [16]:
def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    pattern1 = r'(R)(\d+)'
    processed_text = re.sub(pattern1, r'R \2', input_text)

    pattern2 = r'(\d+)(R)'
    processed_text = re.sub(pattern2, r'\1 R', processed_text)

    pattern3 = r'(\/)(\d+)'
    processed_text = re.sub(pattern3, r'/ \2', processed_text)

    pattern4 = r'(\d+)(\/)'
    processed_text = re.sub(pattern4, r'\1 /', processed_text)

    pattern5 = r'(\\)(\d+)'
    processed_text = re.sub(pattern5, r'\\ \2', processed_text)

    pattern6 = r'(\d+)(\\)'
    processed_text = re.sub(pattern6, r'\1 \\', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    # processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    # processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")
    processed_text = processed_text.replace("(", " ( ")
    processed_text = processed_text.replace(")", " ) ")
    processed_text = processed_text.replace("[", " [ ")
    processed_text = processed_text.replace("]", " ] ")
    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины']
    new_df['height'] = df['Высота профиля']
    new_df['radius'] = df['Диаметр']
    new_df['v_ind'] = df['Индекс нагрузки и скорости']
    new_df['brand'] = df['Unnamed: 29']
    new_df['line'] = df['Модель']

    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Continental)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)
    return new_df

def create_bio_tags(row):
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        # if word == 'R':
        #     continue
        if word == width:
            tags[i] = 'B-width'
        elif word == height:
            tags[i] = 'B-height'
        elif word.upper() in radius.upper():
            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
        elif word in brand:
            tags[i] = 'B-brand'
        elif word.upper() in v_ind.upper():
            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'    

        elif word.upper() in line.upper() or re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")) in line.upper():
            if len(re.sub(r"[^a-zA-Zа-яА-Я]","", word.upper().replace("СПОРТ", "SPORT")))<3 and Begin_line:
                continue
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
        elif word.startswith('Conti') and word[5:] in line:    #   для Continental добавим искусственно
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'

    # Собираем теги в строку
    return ','.join(tags)

def extract_Continental(text):
    brands = re.findall(r'Continental|CONTINENTAL|Континенталь|continental|КОНТИНЕНТАЛЬ', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,[ CONTISPORTCONTACT 5 SUV 23555 R 19 105W XL ...,235,55,R19,105W,[CONTINENTAL],SportContact 5 SUV,"O,O,B-v_ind,B-line,I-line,B-radius,I-radius,I-..."
1,125 / 80 R 13 Continental ContiWinterContact T...,125,80,R13,65Q,[Continental],WinterContact TS800,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
2,14 / 155 / 65 Continental IceContact 3 TA 75T,155,65,R14,75T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
3,14 / 185 / 60 Continental IceContact 3 TA 82T,185,60,R14,82T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
4,145 / 65 R 15 72T Continental VikingContact 7,145,65,R15,72T,[Continental],VikingContact 7,"B-width,O,B-height,B-radius,I-radius,B-v_ind,B..."
...,...,...,...,...,...,...,...,...
20276,Континенталь 295 / 30 / 19 W 100 ContiWinterCo...,295,30,R19,100W,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
20277,Континенталь 295 / 30 / 21 V 102 WinterContact...,295,30,R21,102V,[Континенталь],WinterContact TS860,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
20278,Континенталь 295 / 40 / 19 V 108 ContiWinterCo...,295,40,R19,108V,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
20279,Континенталь 305 / 40 / 20 Y 112 ContiSportCon...,305,40,R20,112Y,[Континенталь],SportContact 5,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."


In [17]:
dict(new_df.line.value_counts())


{'IceContact 3': 2288,
 'VikingContact 7': 1908,
 'SportContact 5': 1370,
 'WinterContact TS860': 1188,
 'PremiumContact 6': 1183,
 'WinterContact TS850': 882,
 'WinterContact TS830': 858,
 'SportContact 6': 765,
 'WinterContact TS870': 612,
 'WinterContact TS850 SUV': 603,
 'EcoContact 6': 560,
 'SportContact 5 SUV': 473,
 'SportContact 7': 417,
 'VanContact Ice': 371,
 'CrossContact LX Sport': 369,
 'WinterContact TS860 RunFlat': 351,
 'CrossContact LX2': 327,
 'IceContact 2 SUV': 296,
 'CrossContact UHP': 295,
 'IceContact 2': 244,
 'SportContact 5 RunFlat': 234,
 'VanContact Winter': 233,
 'VanContact Viking': 228,
 'CrossContact ATR': 209,
 'PremiumContact 5': 207,
 'SportContact 3': 198,
 'Contact 4x4': 176,
 'PremiumContact 6 RunFlat': 166,
 'VikingContact 7 RunFlat': 163,
 'VanContact 100': 157,
 'WinterContact TS810': 152,
 'EcoContact 5': 150,
 'CrossContact Winter': 147,
 'UltraContact': 140,
 'PremiumContact 7': 135,
 'VikingContact 6': 134,
 'WinterContact TS830 SUV': 125,

In [18]:
new_df[new_df.line == 'VanContact A/S']

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
434,195 / 75 R 16 Continental VanContact A/S Ultra...,195,75,R16,110R,[Continental],VanContact A/S,"B-width,O,B-height,B-radius,I-radius,B-brand,B..."
443,195 / 75 R 16C Continental VanContact Ultra A/...,195,75,R16,110R,[Continental],VanContact A/S,"B-width,O,B-height,B-radius,O,B-brand,B-line,O..."
16388,Continental VanContact A/S Ultra 195 / 75 R 16...,195,75,R16,110R,[Continental],VanContact A/S,"B-brand,B-line,I-line,O,B-width,O,B-height,B-r..."
16389,Continental VanContact A/S Ultra 195 / 75 R 16...,195,75,R16,110R,[Continental],VanContact A/S,"B-brand,B-line,I-line,O,B-width,O,B-height,B-r..."
19677,R 16 195 / 75 C 110 / 108 R Continental VanCon...,195,75,R16,110R,[Continental],VanContact A/S,"B-radius,I-radius,B-width,O,B-height,O,B-v_ind..."


In [34]:
import re

def separate_letters_and_numbers(input_string):
    # Используем регулярное выражение для поиска сочетаний букв и цифр
    pattern = re.compile(r'(\D+|\d+)')
    
    # Используем findall для нахождения всех сочетаний
    matches = pattern.findall(input_string)
    print(matches)
    # Возвращаем строку с пробелами между буквами и цифрами
    return ' '.join(matches)

# Пример использования
input_string = "Continental VanContact A/S Ultra 195/75 R16C 110/108R"
result = separate_letters_and_numbers(input_string)

# Вывод результата
print(result)


['Continental VanContact A/S Ultra ', '195', '/', '75', ' R', '16', 'C ', '110', '/', '108', 'R']
Continental VanContact A/S Ultra  195 / 75  R 16 C  110 / 108 R


In [23]:
index = 16389
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Continental VanContact A/S Ultra 195 / 75 R 16C 110 R 

Continental: B-brand
VanContact: B-line
A/S: I-line
Ultra: O
195: B-width
/: O
75: B-height
R: B-radius
16C: I-line
110: B-v_ind
R: I-radius


In [24]:
example

{'offer': 'Continental VanContact A/S Ultra 195 / 75 R 16C 110 R',
 'width': '195',
 'height': '75',
 'radius': 'R16',
 'v_ind': '110R',
 'brand': ['Continental'],
 'line': 'VanContact A/S',
 'BIO_Tags': 'B-brand,B-line,I-line,O,B-width,O,B-height,B-radius,I-line,B-v_ind,I-radius'}

In [21]:
example1 = {'offer': 'Континенталь 305 / 40 / 20 Y 112 SportContact 5 5 XL S 1234567890fsfdh',
 'width': '305',
 'height': '40',
 'radius': 'R20',
 'v_ind': '112Y',
 'brand': ['Континенталь'],
 'line': 'SportContact 5'}

example1_BIO = create_bio_tags(example1)

print(example1['offer'],'\n')
offer = example1['offer'].split()
BIO_Tags = example1_BIO.split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Континенталь 305 / 40 / 20 Y 112 SportContact 5 5 XL S 1234567890fsfdh 

Континенталь: B-brand
305: B-width
/: O
40: B-height
/: O
20: B-radius
Y: B-v_ind
112: I-v_ind
SportContact: B-line
5: I-line
5: I-line
XL: O
S: I-line
1234567890fsfdh: O


In [22]:
label2id = {'B-width': 1,
            'B-height': 2, 
            'B-radius': 3, 
            'I-radius': 4,
            'B-brand': 5, 
            'B-line': 6, 
            'I-line': 7,
            'B-v_ind': 8,
            'I-v_ind': 9,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'I-radius',
            5: 'B-brand', 
            6: 'B-line', 
            7: 'I-line',
            8: 'B-v_ind',
            9: 'I-v_ind',
            0: 'O'}